In [4]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u

c = const.c.value
G = const.G.value
M_sun = const.M_sun.value
M_earth = const.M_earth.value

In [5]:
def rel_len(x_1, x_2):
    r_vec = x_1 - x_2
    r = np.linalg.norm(r_vec)
    return r

def fourdot(V_1, V_2):
    return V_1[0]*V_2[0] - V_1[1]*V_2[1] - V_1[2]*V_2[2]

def findR_S(m_1, m_2, DL):
    if DL == True:
        c = 1
        G = 1
    else:
        c = const.c.value
        G = const.G.value
    R_S = 2 * (m_1 + m_2) * G/c**2
    return R_S

def findrCM(x_1, x_2, m_1, m_2):
    M = m_1 + m_2
    r_cm = (m_1*x_1 + m_2*x_2)/M
    return r_cm

def γ(v, DL):
    if DL == True:
        c = 1
    else:
        c = const.c.value
    return 1/np.sqrt(1 - np.dot(v,v)/c**2)

def three_p(v, m, DL):
    return γ(v, DL)*m*v

def four_P(p, m, DL):
    if DL == True:
        c = 1
    else:
        c = const.c.value
    return np.array([np.sqrt(np.dot(p, p)*c**2 + m**2*c**4)/c, p[0], p[1]])

def rotate(V, θ):
    rot = np.array([[np.cos(θ), -np.sin(θ)], [np.sin(θ), np.cos(θ)]])
    return np.dot(rot, V)

In [6]:
def accelerationN(t, s, m_1, m_2, r_max, DL, R_S, α = 1):
    x_1, y_1, x_2, y_2, vx_1, vy_1, vx_2, vy_2 = s
    x_1, x_2, v_1, v_2 = np.array([x_1, y_1]), np.array([x_2, y_2]), np.array([vx_1, vy_1]), np.array([vx_2, vy_2])

    if DL == True:
        G = 1
    else:
        G = const.G.value
    
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    n = r_vec/r

    c1 = - G*m_2/r**2
    c2 =   G*m_1/r**2

    ax_1, ay_1 = c1*n[0], c1*n[1]
    ax_2, ay_2 = c2*n[0], c2*n[1]
    return vx_1, vy_1, vx_2, vy_2, ax_1, ay_1, ax_2, ay_2

def accelerationPM(t, s, m_1, m_2, r_max, DL, R_S, α = 1):
    x_1, y_1, x_2, y_2, vx_1, vy_1, vx_2, vy_2 = s
    x_1, x_2, v_1, v_2 = np.array([x_1, y_1]), np.array([x_2, y_2]), np.array([vx_1, vy_1]), np.array([vx_2, vy_2])
    
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c.value
        G = const.G.value
    
    P_1, P_2 = punch(v_1, v_2, m_1, m_2, DL)

    if α == 0:
        P_1[0] = m_1*c
        P_2[0] = m_2*c

    P = np.array([P_1, P_2])
    E_1 = P_1[0]*c
    E_2 = P_2[0]*c
    Es = np.array([E_1, E_2])


    def E(i):
        return Es[i-1]

    def E_dot(i):
        return (p(i,1)*p_dot(i,1) + p(i,2)*p_dot(i,2))*c**2/E(i)

    def p(i, σ):
        return P[i-1,σ]

    def p_dot(i, σ):
        result = G*r_vec[σ]/r**3 * (m_1**2*m_2**2*c**4 - 2*fourdot(P_1, P_2)**2)/(E(1)*E(2))
        if i == 1:
            result = result
        elif i == 2:
            result = - result
        else:
            print('Error: i is different than 1 or 2')
        return result

    def P_dot(i):
        return np.array([E_dot(i)/c, p_dot(i, 1),  p_dot(i, 2)])

    def q_dot(i, σ):
        j = 3 - i
        return p(i, σ)*c**2/E(i) - α * G/r * (4*fourdot(P_1, P_2)/(E(1)*E(2)) * (E(j)/E(i)*p(i, σ) - p(j, σ)) + c**2*(m_1**2*m_2**2*c**4 - 2*fourdot(P_1,P_2)**2)/(E(1)*E(2))**2 * E(j)/E(i)*p(i, σ))
    

    def a(i, σ):
        j = 3 - i
        return p_dot(i, σ)*c**2/E(i) + α * ( - p(i, σ)*E_dot(i)*c**2/E(i)**2 + G/r**2*r_dot*(4*fourdot(P_1, P_2)/(E(1)*E(2)) * (E(j)/E(i)*p(i, σ) - p(j, σ)) + c**2*(m_1**2*m_2**2*c**4 - 2*fourdot(P_1,P_2)**2)/(E(1)*E(2))**2 * E(j)/E(i)*p(i, σ)) - G/r * (4*((fourdot(P_dot(1), P_2) + fourdot(P_1, P_dot(2)))/(E(1)*E(2)) - fourdot(P_1, P_2)/(E(1)*E(2))**2*(E_dot(1)*E(2) + E(1)*E_dot(2)))*(E(j)/E(i)*p(i, σ) - p(j, σ)) + 4 * fourdot(P_1, P_2)/(E(1)*E(2)) * ((E_dot(j)/E(i) - E(j)*E_dot(i)/E(i)**2)*p(i, σ) + E(j)/E(i)*p_dot(i, σ) - p_dot(j, σ)) - (4* fourdot(P_1, P_2)*(fourdot(P_dot(1), P_2) + fourdot(P_1, P_dot(2)))/(E(1)*E(2))**2 - 2*(m_1**2*m_2**2*c**4 - 2*fourdot(P_1, P_2)**2)/(E(1)*E(2))**3*(E_dot(1)*E(2) + E(1)*E_dot(2)))*E(j)/E(i)*p(i, σ)*c**2 + (m_1**2*m_2**2*c**4 - 2*fourdot(P_1, P_2)**2)/(E(1)*E(2))**2*( (E_dot(j)/E(i) - E(j)*E_dot(i)/E(i)**2)*p(i, σ) + E(j)/E(i)*p_dot(i, σ))*c**2) )


    r = rel_len(x_1, x_2)
    r_vec = np.array([0, x_1[0] - x_2[0], x_1[1] - x_2[1]])

    v_1 = vx_1, vy_1 = np.array([q_dot(1,1), q_dot(1,2)])
    v_2 = vx_2, vy_2 = np.array([q_dot(2,1), q_dot(2,2)])

    r_dot = (r_vec[1]*(v_1[0] - v_2[0]) + r_vec[2]*(v_1[1] - v_2[1]))/r

    a_1 = ax_1, ay_1 = np.array([a(1, 1), a(1, 2)])
    a_2 = ax_2, ay_2 = np.array([a(2, 1), a(2, 2)])

    print(r, x_1, x_2, v_1, v_2, a_1, a_2)

    return vx_1, vy_1, vx_2, vy_2, ax_1, ay_1, ax_2, ay_2


def boost(v_1, v_2, a_1, a_2, dt):
    v_1 += a_1*dt
    v_2 += a_2*dt
    return np.array([v_1, v_2])

def move(x_1, x_2, v_1, v_2, dt):
    x_1 += v_1*dt
    x_2 += v_2*dt
    return np.array([x_1, x_2])

def punch(v_1, v_2, m_1, m_2, DL):
    p_1 = γ(v_1, DL)*v_1*m_1
    p_2 = γ(v_2, DL)*v_2*m_2
    P_1 = four_P(p_1, m_1, DL)
    P_2 = four_P(p_2, m_2, DL)
    return np.array([P_1, P_2])

In [7]:
from numpy import sqrt, abs

# Funktion som finder længden af rho givet x,y
def rhoFind(x,y):
    rho = sqrt(x**2 + y**2)
    return rho

# Funktion som finder lambda givet x,y,vx,vy
def lamFind(x,y,vx,vy):
    lam = abs(vx*y - vy*x)
    return lam

# Funktion som tager startbetingelserne (s = [rhox, rhoy, vx, vy]) og beregner de tidsafledte bevægelsesligninger (r_max, og R_S er inkluderet pga 'events' i solve_ivp funktionen)
def diffs(t, s, r_max, R_S):
    x,y,vx,vy=s
    rho = rhoFind(x,y)
    lam = lamFind(x,y,vx,vy)
    K = 1 + (3*lam**2)/rho**2
    
    dx = vx
    dy = vy
    dvx = -x/(2*rho**3)*K
    dvy = -y/(2*rho**3)*K
    return [dx, dy, dvx, dvy]

In [8]:
# En funktion der returnerer forskellen mellem rho og R_S givet en tid og en systemtilstand.
def dyk(t, s, m_1, m_2, r_max, DL, R_S=1, α=1):
    x_1, y_1, x_2, y_2, vx_1, vy_1, vx_2, vy_2 = s
    x_1, x_2, v_1, v_2 = np.array([x_1, y_1]), np.array([x_2, y_2]), np.array([vx_1, vy_1]), np.array([vx_2, vy_2])
    r = rel_len(x_1, x_2)
    return r - R_S
dyk.terminal = True

# En funktion der returnerer forskellen på r_max og rho givet en tid og en systemtilstand.
def sprædning(t, s, m_1, m_2, r_max, DL, R_S=1, α=1):
    x_1, y_1, x_2, y_2, vx_1, vy_1, vx_2, vy_2 = s
    x_1, x_2, v_1, v_2 = np.array([x_1, y_1]), np.array([x_2, y_2]), np.array([vx_1, vy_1]), np.array([vx_2, vy_2])
    r = rel_len(x_1, x_2)
    return r_max - r
sprædning.terminal = True

In [9]:
from scipy.integrate import solve_ivp

# Funtkion som returnerer tider, positioner, hastigheder, afstande og integrationstid for orbitaler, givet startværdier og evt en øvre tidsgrænse
def orbitals(s, m_1, m_2, t_max=2000, res = 10000, R_S=1, d=5, DL = False, α = 1, N = False):
    y0 = x_1, y_1, x_2, y_2, vx_1, vy_1, vx_2, vy_2 = s
    r_max = d*rel_len(x_1, x_2)
    t_span = [0,t_max]
    t_eval = np.linspace(t_span[0], t_span[1], res)
    
    if N == False:
        sol = solve_ivp(accelerationPM, t_span = t_span, y0 = y0, t_eval = t_eval, events = [sprædning, dyk], method = 'DOP853', args=(m_1, m_2, r_max, DL, R_S, α))
    else:
        sol = solve_ivp(accelerationN, t_span = t_span, y0 = y0, t_eval = t_eval, events = [sprædning, dyk], method = 'DOP853', args=(m_1, m_2, r_max, DL, R_S, α))
    t = sol.t
    pos1, pos2 = np.array([sol.y[0], sol.y[1]]), np.array([sol.y[2], sol.y[3]])
    vel1, vel2 = np.array([sol.y[4], sol.y[5]]), np.array([sol.y[6], sol.y[7]])
    t_events = sol.t_events
    
    r = rel_len(pos1, pos2)
    
    posCM = []
    for i in range(len(pos1)):
        posCM.append(findrCM(pos1[i], pos2[i], m_1, m_2))

    if len(t_events[0]) > 0:
        t_max = t_events[0][0]
    elif len(t_events[1]) > 0:
        t_max = t_events[1][0]
    else:
        t_max = t_max
    return t, [pos1, pos2, posCM], [vel1, vel2], r, t_max, sol

In [10]:
def findMaxPos(pos):
    xmax, xmin, ymax, ymin = max(pos[0]), min(pos[0]), max(pos[1]), min(pos[1])
    return xmax, xmin, ymax, ymin

def plotLims(pos):
    start_pos = np.array([pos[0][0], pos[1][0]])
    end_pos = np.array([pos[0][-1], pos[1][-1]])

    xmax, xmin, ymax, ymin = findMaxPos(pos)
    max_pos = max(max(abs(start_pos)),max(abs(end_pos)))

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def plotLimsTwoBody(pos1, pos2):
    start_pos1 = np.array([pos1[0][0], pos1[1][0]])
    end_pos1 = np.array([pos1[0][-1], pos1[1][-1]])
    start_pos2 = np.array([pos2[0][0], pos2[1][0]])
    end_pos2 = np.array([pos2[0][-1], pos2[1][-1]])

    x1max, x1min, y1max, y1min = findMaxPos(pos1)
    x2max, x2min, y2max, y2min = findMaxPos(pos2)

    xmax = max(x1max, x2max)
    xmin = min(x1min, x2min)
    ymax = max(y1max, y2max)
    ymin = min(y1min, y2min)

    max_pos1 = max(max(abs(start_pos1)),max(abs(end_pos1)))
    max_pos2 = max(max(abs(start_pos2)),max(abs(end_pos2)))
    max_pos = max(max_pos1, max_pos2)

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def orbPlotterOLD(positions, positionsN = 0, slice = slice(1, -1, 1), aspect = 1, filename='', CM = True, save = False, show = True, DL = False, figsize=(8,8)):
    x_1, x_2, x_cm = positions[0:3,:,slice]
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = np.zeros_like(x_1)

    fig, ax = plt.subplots(figsize=figsize)
    xlim, ylim = plotLimsTwoBody(x_1, x_2)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # ax.plot(x_cm[0], x_cm[1], 'g:', label = 'CM', lw=0.4)
    ax.plot(x_cm[0], x_cm[1], 'g:', lw=0.5)
    ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
    ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    # ax.plot(x_1[0], x_1[1],'b', label = 'm_1', lw=0.75)
    ax.plot(x_1[0], x_1[1],'b', lw=1)
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    # ax.plot(x_2[0], x_2[1],'r', label = 'm_2', lw=0.75)
    ax.plot(x_2[0], x_2[1],'r', lw=1)
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    if type(positionsN) == np.ndarray:
        x_1N, x_2N, x_cmN = positionsN[0:3,:,slice]
        if CM == True:
            x_1N = x_1N - x_cmN
            x_2N = x_2N - x_cmN
            x_cmN = np.zeros_like(x_1N)
        ax.plot(x_1N[0], x_1N[1], 'y:', label='Newtonian motion')

    if DL == True:
        ax.set_xlabel(r'$R_S$', fontsize = 15)
        ax.set_ylabel(r'$R_S$', fontsize = 15)
    else:
        ax.set_xlabel(r'$x \ [ \mathrm{m} ]$', fontsize = 15)
        ax.set_ylabel(r'$y \ [ \mathrm{m} ]$', fontsize = 15)  
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey', fontsize = 12, bbox_to_anchor=(1.01,1), loc='upper left')

    # d = {
    #      'E:':      parameters[0],
    #      '|p_1|:':  parameters[1],
    #      '|p_2|:':  parameters[2],
    #      '|L_1|:':  parameters[3],
    #      '|L_2|:':  parameters[4],
    #      'R_S:':    parameters[5],
    #      'b:':      parameters[6],
    #      'θ:':      parameters[7],
    #     }

    # text = nice_string_output(d, extra_spacing=3, decimals=2)
    # add_text_to_ax(1.04, 0, text, ax, fontsize=12)

    plt.title(f'{filename}', fontsize=25, y=1.08)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/{filename}.png', dpi=300, transparent = False)
    return

def orbPlotter(positions, positionsN = 0, xlim = 0, ylim = 0, slice = slice(0, -1, 1),aspect = 1, filename='', CM = True, save = False, show = True, DL = False, N = False, figsize=(8,8)):
    if DL == False:
        x_1, x_2, x_cm = positions[0:3,:,slice]/149597871000
    else:
        x_1, x_2, x_cm = positions[0:3,:,slice]
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = np.zeros_like(x_1)

    fig, ax = plt.subplots(figsize=figsize)
    if type(xlim) != tuple:
        xlim = plotLimsTwoBody(x_1, x_2)[0]
    if type(ylim) != tuple:
        ylim = plotLimsTwoBody(x_1, x_2)[1]
    
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    if CM == False:
        ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
        ax.plot(x_cm[0][0], x_cm[1][0], 'g.',markersize=8)
        ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b')
    ax.plot(x_1[0][0], x_1[1][0], 'b.', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'bo', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r')
    ax.plot(x_2[0][0], x_2[1][0], 'r.', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'ro', label = 'm_2 stop',markersize=15)

    if type(positionsN) == np.ndarray:
        if DL == False:
            x_1N, x_2N, x_cmN = positionsN[0:3,:,slice]/149597871000
        else:
            x_1N, x_2N, x_cmN = positionsN[0:3,:,slice]
        if CM == True:
            x_1N = x_1N - x_cmN
            x_2N = x_2N - x_cmN
        ax.plot(x_1N[0], x_1N[1], 'y:', label='Newtonian motion')

    if DL == True:
        ax.set_xlabel('$R_S$', fontsize = 15)
        ax.set_ylabel('$R_S$', fontsize = 15)
    else:
        ax.set_xlabel('$x \ [\mathrm{AU}]$', fontsize = 15)
        ax.set_ylabel('$y \ [\mathrm{AU}]$', fontsize = 15)  
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey', fontsize = 12, bbox_to_anchor=(1.01,1), loc='upper left')

    # d = {
    #      'E:':      parameters[0],
    #      '|p_1|:':  parameters[1],
    #      '|p_2|:':  parameters[2],
    #      '|L_1|:':  parameters[3],
    #      '|L_2|:':  parameters[4],
    #      'R_S:':    parameters[5],
    #      'b:':      parameters[6],
    #      'θ:':      parameters[7],
    #     }

    # text = nice_string_output(d, extra_spacing=3, decimals=2)
    # add_text_to_ax(1.04, 0, text, ax, fontsize=12)

    plt.title(f'{filename}', fontsize=25, y=1.08)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'./Plots/{filename}.png', dpi=300, transparent = False)
    return